In [ ]:
import requests
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from covalentcredentials import API_KEY, address

#suppress scientific notation
pd.options.display.float_format = '{:.9f}'.format

## Covalent API

In [ ]:
COV_API_KEY = API_KEY

hidelist = ['FLUX', 'EVER', 'ARKR.org', 'FF18.io', 'Swap7.org', 'Money', 'FATDOGE', 'FBONK', 'H3X', 'XCH5.io', 'PEBO',
         'AGMC.io', 'GUH', 'Def8.io', 'BeeToken', 'SHIBO', 'DCAT']

#### Get Wallet Balances

In [ ]:
def get_wallet(address, API_KEY, hidelist):
    wallet = []
    balance_url = 'https://api.covalenthq.com/v1/56/address/{}/balances_v2/?quote-currency=USD&key={}'.format(address, API_KEY)
    resp = json.loads(requests.get(balance_url).text)
    for i in resp['data']['items']:
        if i['contract_ticker_symbol'] in hidelist:
            pass
        else:
            decimal = int(i['contract_decimals'])
            coin = {'Name': i['contract_ticker_symbol'], 'Contract': i['contract_address'],'Balance': float(i['balance'])*(10**(-decimal)), 
                'Price': i['quote_rate'], 'USD Value': i['quote']}
            wallet.append(coin)
    return wallet

In [ ]:
wallet = get_wallet(address, COV_API_KEY, hidelist)

#### Get Current Prices from PancakeSwap

In [ ]:
def get_prices(wallet):
    base_url = 'https://api.pancakeswap.info/api/v2/tokens/'
    updatedprices = []
    for i in wallet:
        token_address = i['Contract']
        getprice = base_url + str(token_address)
        print(getprice)
        resp = json.loads(requests.get(getprice).text)
        print(resp)
        try:
            price = resp['data']['price']
            entry = {'Name': i['Name'], 'Price': price}
            updatedprices.append(entry)
        except KeyError:
            pass
        
    return updatedprices

In [ ]:
current_prices = get_prices(wallet)

In [ ]:
zerodrop = []
for i in current_prices:
    if float(i['Price']) == 0:
        if i['Name'] != 'BNB':
            zerodrop.append(i['Name'])

In [ ]:
for i in wallet:
    for j in current_prices:
        if j['Name'] == i['Name']:
            i['Price'] = j['Price']

#### Get Transactions

In [ ]:
# Loops through all transactions of each token. Gathers price of bnb at time of transaction * quantity of bnb purchase. 
# Returns total net cost (minus gas fees) after summing USD value of all transactions for a given token
def get_transactions(wallet, address, API_KEY):
    netcostlist = []
    for i in wallet:
        print(i['Name'])
        transactions = {}
        contract = i['Contract']
        time.sleep(1)
        txurl = 'https://api.covalenthq.com/v1/56/address/{}/transfers_v2/?quote-currency=USD&format=JSON&contract-address={}&key={}'.format(address, contract, API_KEY)
        txresp = json.loads(requests.get(txurl).text)
        if txresp['data'] is not None:
            txtotal = []
            for j in txresp['data']['items']:
                cut = j['block_signed_at'].split("T")[0]
                cutformat = pd.to_datetime(cut)
                date = datetime.datetime.strftime(cutformat, "%d-%m-%Y")
                bnbusd = round(json.loads(requests.get('https://api.coingecko.com/api/v3/coins/binancecoin/history?date={}'.format(date)).text)['market_data']['current_price']['usd'], 2)
                time.sleep(1)
                dollarvalue = (int(j['value'])*(1E-18)) * int(bnbusd)
                txtotal.append(dollarvalue)
        else:
            txtotal = [0]
        netcost = {'Name': i['Name'], 'Net Cost': sum(txtotal)}
        netcostlist.append(netcost)
    return netcostlist

In [ ]:
netcost = get_transactions(wallet, address, COV_API_KEY)

In [ ]:
print(netcost)

In [ ]:
# Update wallet list of dicts with net cost value
for i in wallet:
    for j in netcost:
        if j['Name'] == i['Name']:
            i['Net Cost'] = j['Net Cost']

In [ ]:
walletdf = pd.DataFrame(wallet)
walletdf = walletdf[walletdf['Price'].notna()]
walletdf

In [ ]:
walletdf['Price'] = walletdf['Price'].astype(float)
walletdf['Balance'] = walletdf['Balance'].astype(float)
walletdf['USD Value'] = walletdf['Balance'] * walletdf['Price']

In [ ]:
walletdf['Pct Gain'] = ((walletdf['USD Value'] - walletdf['Net Cost'])/walletdf['Net Cost'])*100

In [ ]:
walletdf[~walletdf['Name'].isin(zerodrop)]

#### Historical Prices

In [ ]:
from datetime import datetime, timedelta
import numpy as np

weekstart = (datetime.today() - timedelta(days=7)).strftime("%Y-%m-%d")
weekend = datetime.today().strftime("%Y-%m-%d")
sixweek = (datetime.today() - timedelta(days=42)).strftime("%Y-%m-%d")
addresslist = list(walletdf['Contract'])

In [ ]:
# 7 day moving average
def get_week_MA(addresslist, start, end, COV_API_KEY):
    weekpricelist=[]
    for address in addresslist:
        tmp = []
        try:
            weekurl = 'https://api.covalenthq.com/v1/pricing/historical_by_addresses_v2/56/USD/{}/?quote-currency=USD&format=JSON&from={}&to={}&key={}'.format(address,start,end, COV_API_KEY)
            resp = json.loads(requests.get(weekurl).text)
            print(resp['data'])
            time.sleep(1)
            for i in range(7):
                tmp.append(resp['data']['prices'][0]['prices'][i]['price'])
            weekavg = np.average(tmp)
            entry = {'Name': resp['data']['prices'][0]['prices'][7]['contract_metadata']['contract_ticker_symbol'], '7-Day MA': weekavg}
            weekpricelist.append(entry)
            print(resp['data']['prices'][0]['prices'][7]['contract_metadata']['contract_ticker_symbol'])
        except IndexError:
            entry = {'Name': resp['data']['prices'][0]['contract_ticker_symbol'], '7-Day MA': 'NaN'}
            weekpricelist.append(entry)
            print(resp['data']['prices'][0]['contract_name'])
    return weekpricelist

weekprices_MA = get_week_MA(addresslist, weekstart, weekend, COV_API_KEY)

In [ ]:
weekprices_MA
week_MA_df = pd.DataFrame(weekprices_MA)
walletdf['7 Day Moving Average'] = week_MA_df['7-Day MA']

In [ ]:
# 6 week Moving Average
def get_6week_MA(addresslist, start, end, COV_API_KEY):
    weekpricelist=[]
    for address in addresslist:
        tmp = []
        try:
            weekurl = 'https://api.covalenthq.com/v1/pricing/historical_by_addresses_v2/56/USD/{}/?quote-currency=USD&format=JSON&from={}&to={}&key={}'.format(address,start,end, COV_API_KEY)
            resp = json.loads(requests.get(weekurl).text)
            time.sleep(1)
            for i in range(42):
                tmp.append(resp['data']['prices'][0]['prices'][i]['price'])
            weekavg = np.average(tmp)
            entry = {'Name': resp['data']['prices'][0]['prices'][7]['contract_metadata']['contract_ticker_symbol'], '6 Week MA': weekavg}
            weekpricelist.append(entry)
            print(resp['data']['prices'][0]['prices'][7]['contract_metadata']['contract_ticker_symbol'])
        except IndexError:
            entry = {'Name': resp['data']['prices'][0]['contract_ticker_symbol'], '6 Week MA': 'NaN'}
            weekpricelist.append(entry)
            print(resp['data']['prices'][0]['contract_name'])
    return weekpricelist

sixweeks_MA = get_6week_MA(addresslist, sixweek, weekend, COV_API_KEY)
print(sixweeks_MA)

In [ ]:
sixweeks_MA_df = pd.DataFrame(sixweeks_MA)
sixweeks_MA_df
walletdf['6 Week Moving Average'] = sixweeks_MA_df['6 Week MA']

In [ ]:
walletdf = walletdf[walletdf['Name'].str.contains("None") == False]

In [ ]:
# Filter Pct Gain Column in the green or red

def color_negative_red(val):
    color = 'red' if val < 0 else 'green'
    return f'color: {color}'

def highlight_cell(val):
    if val > 0:
        color = 'green'
    else:
        color = 'red'
    return f'background-color: {color}'

walletdf.style.applymap(highlight_cell, subset=['Pct Gain'])

In [ ]:
walletdf.reset_index(drop=True, inplace=True)
walletdf

### Generate Figures from Dataframe

In [ ]:
import os
import pathlib

dist = walletdf.groupby(['Name']).sum().plot(kind = 'pie', y = 'USD Value', autopct = '%1.0f%%', pctdistance = .8, labeldistance = 1, figsize = (7,7), legend=False, title = "Distribution of Funds in USD")
fig1 = dist.figure
cwd = os.getcwd()
folder = r'\results\distfunds.png'
path_plot = str(cwd) + folder
print(path_plot)
fig1.savefig(path_plot, dpi = fig1.dpi, bbox_inches = "tight")

In [ ]:
neg = []
for index, row in walletdf.iterrows():
    if row['Pct Gain'] < 0:
        neg.append('r')
    else:
        neg.append('g')
        
ax =  plt.subplot()
ax.set_xlabel('Coin')
ax.set_ylabel('Percentage Gain')
ax.set_xticks(list(range(0, len(walletdf.index))))
ax.set_xticklabels(list(walletdf.Name), rotation= 90)
fig2 = plt.bar(walletdf.index, walletdf['Pct Gain'], color = neg)


cwd = os.getcwd()
folder = r'\results\pctgain.png'
path_plot = str(cwd) + folder
print(path_plot)
ax.figure.savefig(path_plot, bbox_inches = "tight")

### Send Summary to Email

In [ ]:
import smtplib, ssl
from automailcreds import mailpassword, sender_email, receiver_email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.message import EmailMessage
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage

#Email setup
port = 465
smtp_server = 'smtp.gmail.com'
sender = sender_email
receiver = receiver_email
password = mailpassword

context = ssl.create_default_context()

In [ ]:
conv_df = walletdf.to_html()
htmlmsg = MIMEText(conv_df, 'html')

In [ ]:
def send_email(path_plot, smtp_server, smtp_port, from_mail, from_password, to_mail):
    images = [cwd + r'\results\pctgain.png',
             cwd + r'\results\distfunds.png']
    todaydate = str(datetime.today().strftime("%B %d, %Y"))
    # Create the email message
    msg = MIMEMultipart()
    msg['Subject'] = f'Crypto Portfolio Update - {todaydate}'
    msg['From'] = sender
    msg['To'] = receiver

    # Open the files in binary mode and attach to mail
    
    for num,filename in enumerate(images):
        fp = open(filename, 'rb')
        msg_img = MIMEImage(fp.read())
        fp.close()
        msg_img.add_header('Content-ID', '<{}>'.format(num))
        msg_img.add_header('Content-Disposition', 'inline', filename=filename)
        msg.attach(msg_img)
        
        
    # Attach HTML body
    msg.attach(MIMEText(
        f'''
        <html>
            <body>
                <h1 style="text-align: center;">Daily BSC Wallet Update - {todaydate}</h1>
                <h2 style="text-align: center;">Today's data summarized in a table:</h2>
                {htmlmsg}
                <p>This chart shows the distribution of BSC coin funds.</p>
                <p><img src="cid:1"></p>
                <p>This chart shows the percentage gain or loss for each coin (green = gain, red = loss).</p>
                <p><img src="cid:0"></p>
            </body>
        </html>'
        ''',
        'html', 'utf-8'))

    # Send mail
    server = smtplib.SMTP_SSL(smtp_server, port)
    server.ehlo()
    server.login(sender, password)

    server.sendmail(sender, receiver, msg.as_string())
    server.quit()


send_email(path_plot, smtp_server, port, sender, password, receiver)